In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Importar tqdm para ver el progreso la ejecución
from tqdm import tqdm

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [12]:
# Prueba con find.elements()

movie_list = []
movie_tuple = ()

# función para importar el archivo csv y convertirlo a DF
def import_csv(csv):
    # - Importar los datos del CSV de las películas y convertirlo a dataframe
    df_completo = pd.read_csv(csv)
    # - filtrar solo las películas.
    df_movies = df_completo[df_completo["type"].isin(["Movie", "Short"])]
    print("We have imported the movies and shorts of the csv file")
    return df_movies

# función para inicializar el driver y rechazar cookies
def init_driver(url):
    driver = webdriver.Chrome()
    driver.get(url)
    sleep(2)
    driver.find_element("css selector", '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
    print("Hemos rechazado las cookies")
    return driver 

#funcion para recoger toda la información y almacenarla en una tupla
id_list = []
def collect_data(driver, id):
    if id in id_list: # condicional por si la pelicula ya está recogida, ignora los duplicados
        pass # chatty aquí tiene return None, y no pone el else: Supongo que simplifica el código
    else:
        driver.find_element("xpath", '//*[@id="suggestion-search"]').send_keys(id, Keys.ENTER)
        print("We have accessed the movie")
        sleep(3) #dar tiempo para que cargue la pagina

        # si no está la pag de la peli maximizada no se ve el plot completo y solo extrae lo que se ve, que muchas veces incluye ...Read all
        driver.maximize_window() 

        
        # Empezar a recoger información

        #Rating
        try:
            IMDB_rating = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text
            print("We have accessed to the IMDB_rating")
            sleep(2)
        except:
            if not IMDB_rating: # en caso de que no haya puntuación
                IMDB_rating = None
                print("We could not access the rating")                         

        # directoras. BTW: me parece que se pueden contar con los dedos de una mano...
        directors = []
        all_directors = driver.find_elements('xpath', '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li/a')

        for i in range(len(all_directors)):
            directors.append(all_directors[i].get_attribute('innerHTML'))
    
       
        # guionistas    
        writers = []
        all_writers = driver.find_elements("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/div[2]/div/ul/li[2]/div/ul/li/a')
        for i in range(len(all_writers)):
            writers.append(all_writers[i].get_attribute('innerHTML'))                   

        # argumento
        try: 
            plot = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p').text
            print("We have accessed to the plot")
            sleep(2)
        except:
            if not plot:
                plot = None
                print("We could not access the plot ")
            elif plot == "Add a plot":
                plot = None
                print("There was no plot described")

        # duración de la peli
                  
        year = None
        rate_age = None
        lenght = None
        all_others_data = driver.find_elements('xpath', '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li')
        if len(all_others_data) == 2:
            year = all_others_data[0].find_element('css selector','a').get_attribute('innerHTML')
            lenght = all_others_data[1].get_attribute('innerHTML')
        elif len(all_others_data) == 3:
            year = all_others_data[0].find_element('css selector','a').get_attribute('innerHTML')
            rate_age = all_others_data[1].find_element('css selector','a').get_attribute('innerHTML')
            lenght = all_others_data[2].get_attribute('innerHTML')
        else:
            print(f'REVISAR - La pelicula con id {id} tiene {len(all_others_data)} variables')
        # nombre
        try:
            name = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/h1/span').text
            print(f"We have accessed to the movie name - {name}")
            sleep(5)
        except:
            if not name:
                name = None
                print("We could not access the movie name")
        
        # añadir el idOwner de esta película a la lista, para que no la repita
        id_list.append(id)

        movie_tuple = (id, IMDB_rating, directors, writers, plot, lenght, name, year,rate_age)
        sleep(2)
        return movie_tuple
    

#funcion para añadir cada tupla a la lista de películas
def add_movie_tuple(movie_tuple):
    if movie_tuple in movie_list:
        pass
    else:
        movie_list.append(movie_tuple)
    return movie_list

# función para ciclar sobre los idOwner de df_movies
def loop_movies(driver, df_movies):
    for id in df_movies["idOwner"][:100]: #Se reduce el buble en 100 peliculas para no tardar tanto en la estraccion
        movie_tuple = collect_data(driver, id)
        add_movie_tuple(movie_tuple)

    


In [13]:
# Bloque para Ejecutar todas las funciones de arriba
url = "https://www.imdb.com/?ref_=nv_home"
csv = "Team1_API_extraction.csv"
driver = init_driver(url)
df_movies = import_csv(csv)
loop_movies(driver, df_movies)

Hemos rechazado las cookies
We have imported the movies and shorts of the csv file
We have accessed the movie
We have accessed to the IMDB_rating
We have accessed to the plot
We have accessed to the movie name - A todo riesgo
We have accessed the movie
We have accessed to the IMDB_rating
We have accessed to the plot
We have accessed to the movie name - Three Came to Kill
We have accessed the movie
We have accessed to the IMDB_rating
We have accessed to the plot
We have accessed to the movie name - Las dos caras del destino
We have accessed the movie
We have accessed to the IMDB_rating
We have accessed to the plot
We have accessed to the movie name - El círculo rojo
We have accessed the movie
We have accessed to the IMDB_rating
We have accessed to the plot
We have accessed to the movie name - El robo al banco de Inglaterra
We have accessed the movie
We have accessed to the IMDB_rating
We have accessed to the plot
We have accessed to the movie name - La Llorona
We have accessed the movie

KeyboardInterrupt: 

In [14]:
# comprobar qué información se recogió
for tuple in movie_list:
    print(tuple)


('tt0052698', '7,5', ['Claude Sautet'], ['José Giovanni', 'Claude Sautet', 'Pascal Jardin'], 'Un despiadado criminal huye de la persecución, lo que implica cada vez más víctimas.', '1h 50min', 'A todo riesgo', '1960', 'Not Rated')
('tt0053357', '5,5', ['Edward L. Cahn'], ['Robert E. Kent', 'Orville H. Hampton'], 'Tres asesinos profesionales irrumpen en la casa de un empleado del aeropuerto para disparar al avión en el que un primer ministro asiático debe abandonar Estados Unidos.', '1h 11min', 'Three Came to Kill', '1960', 'Approved')
('tt0052661', '6,1', ['Alfred Weidenmann'], ['Herbert Reinecker', 'Igor Sentjurc'], 'Añade un argumento en tu idioma', '1h 32min', 'Las dos caras del destino', '1960', None)
('tt0053228', '6,2', ['Jürgen Roland'], ['Edgar Wallace', 'Egon Eis', 'Wolfgang Menge'], 'Añade un argumento en tu idioma\nA strange, red circle appears on the neck of a man saved from the guillotine. What is its mysterious meaning? Tragically, it turns out to be something of a family

In [17]:
df_resultados = pd.DataFrame(movie_list, columns =["idOwner","IMDB_rating","direction","writers","plot","length","rate_age","year","name"])
df_resultados

,idOwner,IMDB_rating,direction,writers,plot,length,rate_age,year,name
0,tt0052698,"7,5",[Claude Sautet],"[José Giovanni, Claude Sautet, Pascal Jardin]","Un despiadado criminal huye de la persecución,...",1h 50min,A todo riesgo,1960,Not Rated
1,tt0053357,"5,5",[Edward L. Cahn],"[Robert E. Kent, Orville H. Hampton]",Tres asesinos profesionales irrumpen en la cas...,1h 11min,Three Came to Kill,1960,Approved
2,tt0052661,"6,1",[Alfred Weidenmann],"[Herbert Reinecker, Igor Sentjurc]",Añade un argumento en tu idioma,1h 32min,Las dos caras del destino,1960,None
3,tt0053228,"6,2",[Jürgen Roland],"[Edgar Wallace, Egon Eis, Wolfgang Menge]","Añade un argumento en tu idioma\nA strange, re...",1h 32min,El círculo rojo,1960,None
4,tt0053752,"6,6",[John Guillermin],"[John Brophy, Howard Clewes, Richard Maibaum]","Añade un argumento en tu idioma\nIn 1901, a gr...",1h 25min,El robo al banco de Inglaterra,1960,Approved
5,tt0051874,"5,8",[René Cardona],"[Adolfo Torres Portillo, Carmen Toscano]",Felipe y Margarita se casan y tienen al pequeñ...,1h 15min,La Llorona,1960,None
6,tt0053063,"5,9",[Etienne Périer],"[Pierre Boileau, Thomas Narcejac, Dominique Fa...",Añade un argumento en tu idioma\n(1961) Daniel...,1h 33min,Meurtre en 45 tours,1960,None
7,tt0052997,"7,2",[Basil Dearden],"[Bryan Forbes, John Boland]",Un veterano descontento recluta a un grupo de ...,1h 56min,La Liga de los caballeros,1960,Unrated
8,None,None,None,None,None,None,None,None,None
9,tt0053979,"6,3",[Luis Alcoriza],[Luis Alcoriza],Añade un argumento en tu idioma\nLorenzo Gómez...,1h 40min,Los jóvenes,1961,None


In [19]:
#Convertimos en CSV
df_resultados.to_csv("film_details.csv",index=False)